In [1]:
import os

import numpy as np
import pandas as pd

from src import direcslist, nb_setup, raw_data_path

nb_setup()
os.makedirs("Figs", exist_ok=True)

%matplotlib inline

In [6]:
basepath = raw_data_path + "/Imaging/two_cell_asymmetry"
paths = direcslist(
    basepath,
    3,
    exclude=[
        "n2",
    ],
)
lines = [p.split("/")[-2].split("_")[1] for p in paths]
conds = [p.split("/")[-2].split("_")[2] for p in paths]
line_to_genotype = {
    "nwg376": "GCN4",
    "nwg240": "C56S",
    "nwg192": "R183-5A",
    "lp637": "WT",
    "nwg398": "L109R, GCN4",
    "nwg338": "L109R",
}
genotypes = [line_to_genotype[l] for l in lines]
rois_p1 = [np.loadtxt(p + "/ROI_fit_p1.txt") for p in paths]
rois_ab = [np.loadtxt(p + "/ROI_fit_ab.txt") for p in paths]

In [7]:
def area(corners):
    # shoelace method
    n = len(corners)  # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

In [12]:
areas_asi = []
for p1, ab in zip(rois_p1, rois_ab):
    area_ab = area(ab)
    area_p1 = area(p1)
    areas_asi.append(area_ab / (area_ab + area_p1))

In [15]:
df = pd.DataFrame.from_dict(
    {
        "Path": paths,
        "Line": lines,
        "Genotype": genotypes,
        "Condition": conds,
        "Two cell asymmetry": areas_asi,
    }
)
df["Path"] = df["Path"].apply(lambda x: x[len(raw_data_path) :])
df.to_csv("../../../data/2cellASI.csv")